# Evaluate the output of the targetless_extrinsics step
After switching from using the intrinsics database intrinsics to using the corrected calibration table intrinsics, we want to make sure that the estimated pairwise transformations are still very similar to the ones in the current databricks table. This notebook will compare the output of the targetless_extrinsics step when using the intrinsics database intrinsics (current version) and the corrected calibration table intrinsics (new to be merged version).

In [19]:
from wayve.services.data.lakehouse.common.external_access.wayve_delta_table import WayveDeltaTable
from collections import defaultdict
import numpy as np
from scipy.spatial.transform import Rotation as R

In [20]:
TARGETLESS_EXTRINSICS_MAIN_CLOUD_TABLE = "abfss://databricks-users@wayvedevdataset.dfs.core.windows.net/pipeline_testing_tmp/data-lakehouse-jobs-slam-test8e6fa/databricks/delta/inferred__state/calibration_sensor_pose"
TARGETLESS_EXTRINSICS_BRANCH_CLOUD_TABLE = "abfss://databricks-users@wayvedevdataset.dfs.core.windows.net/pipeline_testing_tmp/data-lakehouse-jobs-slam-test55988/databricks/delta/inferred__state/calibration_sensor_pose"

In [21]:
# Read in output of the updated step
def get_output_table(table_path: str):
    df = WayveDeltaTable(path=table_path).to_pandas_dataframe()
    return df

# Read in the output of the original step, which is the "inferred__state.calibration_sensor_pose" table
def get_original_camera_transforms():
    df = WayveDeltaTable.from_database_and_table_name("inferred__state", "calibration_sensor_pose").to_pandas_dataframe()
    return df

# Do inner join on run_id, source and destination frames
def get_merged_main_and_branch_table():
    columns_to_drop = ["_pipeline__vehicle_platform_id", "run_date_iso", "run_time", "run_vehicle_id", "run_label"]
    main_cloud_df = get_output_table(TARGETLESS_EXTRINSICS_MAIN_CLOUD_TABLE).drop(columns=columns_to_drop)
    branch_cloud_df = get_output_table(TARGETLESS_EXTRINSICS_BRANCH_CLOUD_TABLE).drop(columns=columns_to_drop)
    merged_df = main_cloud_df.merge(branch_cloud_df, on=["run_id", "frame_source", "frame_dest"], suffixes=("_main", "_branch"))
    return merged_df

# For each row calculate T_eval = T_original * T_updated^-1, calculate the norm of the translation and the abs angle of the rotation
def calculate_table_diffs():
    merged_df = get_merged_main_and_branch_table()

    dd = defaultdict(list)

    for _, row in merged_df.iterrows():
        dd["run_id"].append(row["run_id"])
        dd["frame_source"].append(row["frame_source"])
        dd["frame_dest"].append(row["frame_dest"])
        T_main = row["transform_main"].reshape((4, 4))
        T_branch = row["transform_branch"].reshape((4, 4))
        T_diff = T_main @ np.linalg.inv(T_branch)

        dd["angle_diff (deg)"].append(R.from_matrix(T_diff[:3, :3]).magnitude() * 180 / np.pi)
        dd["translation_diff (m)"].append(np.linalg.norm(T_diff[:3, 3]))
        
        dd["number_of_features_for_calibration_diff"].append(row["number_of_features_for_calibration_branch"] - row["number_of_features_for_calibration_main"])
        dd["mean_reprojection_error_diff"].append(row["mean_reprojection_error_branch"] - row["mean_reprojection_error_main"])

    diff_df = pd.DataFrame(dd)

    return diff_df

diff_df = calculate_table_diffs()
diff_df

,run_id,frame_source,frame_dest,angle_diff (deg),translation_diff (m),number_of_features_for_calibration_diff,mean_reprojection_error_diff
0,neptune/2023-02-10--15-08-24--session_2022_12_...,front-left-leftward,front-forward,0.004351,0.000042,25,-0.000923
1,neptune/2023-02-10--15-08-24--session_2022_12_...,front-right-rightward,front-forward,0.003823,0.000037,36,-0.003346
2,neptune/2023-02-10--15-08-24--session_2022_12_...,back-right-rightward,front-right-rightward,0.041443,0.001267,-186,-0.097466
3,neptune/2023-02-10--15-08-24--session_2022_12_...,back-left-leftward,front-left-leftward,0.009547,0.000291,-77,0.065637
4,neptune/2023-02-10--15-08-24--session_2022_12_...,back-backward,back-right-rightward,0.015408,0.000126,2,0.014023
5,neptune/2023-02-10--15-08-24--session_2022_12_...,back-backward,back-left-leftward,0.006736,0.000058,-25,0.050563
6,sedna/2023-09-20--08-58-40--session_2023_08_27...,back-backward,back-right-rightward,0.000436,0.000003,-36,0.000068
7,sedna/2023-09-20--08-58-40--session_2023_08_27...,back-right-rightward,front-right-rightward,0.002398,0.000073,214,0.008122
8,sedna/2023-09-20--08-58-40--session_2023_08_27...,back-backward,back-left-leftward,0.010510,0.000076,-98,0.034716
9,sedna/2023-09-20--08-58-40--session_2023_08_27...,front-right-rightward,front-forward,0.000206,0.000002,315,0.000514
